# NV Saturation measurement

Required modules
- powermeterlogic
- laserlogic

In [48]:
import os
import time
import matplotlib.pyplot as plt

from collections import OrderedDict
import datetime

from logic.generic_logic import GenericLogic

In [69]:
class SaturationLogic(GenericLogic):
    def __init__(self):
        self.count_freq = counterlogic._count_frequency
        self.laserlogic = laserlogic
        self.powermeterlogic = powermeterlogic
        self.counterlogic = counterlogic
        self.savelogic = savelogic
        
        self.data = {}
        self.start_power = 0
        self.final_power = 0
        self.num_points = 0
        self.counts_average_from_end = 0
        
    def draw_figure(self):
        powers, counts = self.data["Powermeter power (W)"], self.data["Avg. counts"]

        plt.style.use(savelogic.mpl_qudihira_style)

        fig, ax = plt.subplots()

        ax.errorbar(powers * 1e6, counts, yerr=self.data["Count errors"], capsize=3, fmt="o-")
        ax.set_xlabel("Powermeter power (uW)")
        ax.set_ylabel("Mean counts (kcps)")
        
        fig.tight_layout()
        return fig

    def save_data(self, tag=None):
        timestamp = datetime.datetime.now()

        parameters = OrderedDict()
        
        data_to_save = OrderedDict()
        data_to_save["Laser power (W)"] = self.data["Laser power (W)"]
        data_to_save["Powermeter power (W)"] = self.data["Powermeter power (W)"]
        data_to_save["UHV objective power (W)"] = self.data["UHV objective power (W)"]
        data_to_save["RT objective power (W)"] = self.data["RT objective power (W)"]
        data_to_save["Avg. counts"] = self.data["Avg. counts"]
        data_to_save["Count errors"] = self.data["Count errors"]

        filepath = self.savelogic.get_path_for_module(module_name="Saturation")

        if tag:
            filelabel = f"{tag}_saturation"
        else:
            filelabel = "saturation"
        
        self.log.info('Saturation saved to:\n{0}'.format(filepath))
        
        fig = self.draw_figure()

        self.savelogic.save_data(data, filepath=filepath, parameters=parameters, filelabel=filelabel, 
                                 timestamp=timestamp, plotfig=fig, delimiter='\t')
        
    def set_up_measure(self, start_power, final_power, num_points, counts_average_from_end):
        self.start_power = start_power
        self.final_power = final_power
        self.num_points = num_points
        self.counts_average_from_end = counts_average_from_end
        
    def start_measure(self):
        self.data = {}
        laser_powers = np.linspace(self.start_power, self.final_power, self.num_points)
        counts = np.zeros(self.num_points, dtype=int)
        powermeter_powers = np.zeros(self.num_points)
        count_errors = np.zeros((2, self.num_points))
        
        current_power = self.laserlogic.laser_power

        for idx, laser_power in enumerate(laser_powers):
            try: 
                self.laserlogic.set_power(laser_power)
            except ValueError:
                time.sleep(1)
                self.laserlogic.set_power(laser_power)
            
            # Sleep for 5s
            time.sleep(150 / self.count_freq)
            self.powermeterlogic.get_power(state=None)
            powermeter_powers[idx] = self.powermeterlogic.power
            
            count_array = self.counterlogic.countdata_smoothed[0, -self.counts_average_from_end:]
            counts[idx] = np.mean(count_array)
            count_errors[0, idx] = np.max(count_array) - counts[idx]
            count_errors[1, idx] = counts[idx] - np.min(count_array)

            print(f"[{idx+1}/{len(laser_powers)}] laser power={laser_power * 1e3} mW, powermeter power={powermeter_powers[idx] * 1e6:.2f} uW, counts={counts[idx]}")
        
        self.data = {
            "Laser power (W)": laser_powers, 
            "Powermeter power (W)": powermeter_powers, 
            "UHV objective power (W)": powermeter_powers * powermeterlogic.calibration_param_uhv, 
            "RT objective power (W)": powermeter_powers * powermeterlogic.calibration_param_rt,
            "Avg. counts": counts,
            "Count errors": count_errors
        }

In [70]:
sat = SaturationLogic()
sat.set_up_measure(start_power=5e-3, final_power=100e-3, num_points=10, counts_average_from_end=75)
sat.start_measure()
sat.save_data(tag="NV")

[1/10] laser power=5.0 mW, powermeter power=0.01 uW, counts=232
[2/10] laser power=15.555555555555555 mW, powermeter power=0.56 uW, counts=5804
[3/10] laser power=26.111111111111114 mW, powermeter power=2.42 uW, counts=18419
[4/10] laser power=36.666666666666664 mW, powermeter power=3.79 uW, counts=25904
[5/10] laser power=47.22222222222222 mW, powermeter power=5.51 uW, counts=34284
[6/10] laser power=57.77777777777778 mW, powermeter power=7.85 uW, counts=41192
[7/10] laser power=68.33333333333334 mW, powermeter power=10.35 uW, counts=48429
[8/10] laser power=78.8888888888889 mW, powermeter power=12.72 uW, counts=55197
[9/10] laser power=89.44444444444446 mW, powermeter power=14.74 uW, counts=60992
[10/10] laser power=100.0 mW, powermeter power=16.68 uW, counts=66497
